In [1]:
import pandas as pd
import json

In [145]:
data = []
emotions = ["happy", "sad", ["disgust", "disgust2"], "angry", "fear", "surprise"]
lim = 2840
for emo in emotions:
    if isinstance(emo, list):
        tmp = []
        for d in emo:
            with open("{}.json".format(d), "r") as f:
                tmp.append(json.load(f))
        data.append(sum(tmp, [])[:lim])
    elif isinstance(emo, str):
        with open("{}.json".format(emo), "r") as f:
            if emo == "sad":
                tmp = json.load(f)
                data.append([d for d in tmp if "Love_McD" not in d['user']][:lim])
            else:
                data.append(json.load(f)[:lim])

In [146]:
labels = [i for i in range(len(emotions))]

In [147]:
dfs = []
for l, d in zip(labels, data):
    tmp = pd.DataFrame(d)
    tmp['label'] = l
    dfs.append(tmp)
df = pd.concat(dfs)
df.head()

,fullname,html,id,likes,replies,retweets,text,timestamp,url,user,label
0,himitan,"<p class=""TweetTextSize js-tweet-text tweet-te...",292420866954891265,0,0,0,@yukyan_MNsy おはようありがとう〜ハンバーグまだあるよ食べにおいで（笑）,2013-01-18T23:59:02,/himitan1109/status/292420866954891265,himitan1109,0
1,himitan,"<p class=""TweetTextSize js-tweet-text tweet-te...",292420700034199552,0,0,0,@gdgk0x ありがとういらなかったけど（笑）明日もあるけどさすがに飽きるよね（笑）,2013-01-18T23:58:22,/himitan1109/status/292420700034199552,himitan1109,0
2,ゆかちん,"<p class=""TweetTextSize js-tweet-text tweet-te...",292420678207033344,0,0,0,@kuwamann0410 おはようステキな目覚めだね♪,2013-01-18T23:58:17,/babys_yuka/status/292420678207033344,babys_yuka,0
3,コミー♪,"<p class=""TweetTextSize js-tweet-text tweet-te...",292420470807068672,0,1,0,@andy_yuki ユキちゃん、アンディくんおはよう朝からポカポカなお日様を受けてる4ワン...,2013-01-18T23:57:27,/chokotan82/status/292420470807068672,chokotan82,0
4,業平心,"<p class=""TweetTextSize js-tweet-text tweet-te...",292420452163416064,0,0,0,今空&田んぼだよー pic.twitter.com/HqZQH1B1,2013-01-18T23:57:23,/ncocoro/status/292420452163416064,ncocoro,0


In [148]:
df.shape

(17040, 11)

In [167]:
from IPython.core.display import display, HTML
for i in range(5):
    display(HTML(df.iloc[i]['html']))

In [149]:
import MeCab
from sklearn.utils import shuffle
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [150]:
df = shuffle(df)

X = df['text'].tolist()
y = df['label'].tolist()
tagger = MeCab.Tagger("-Owakati")

X = list(map(tagger.parse, X))

In [151]:
class  TfIdfWord2VecVectorizer:
    def __init__(self):
        pass

    def fit(self, X, y):
        self.tfidf_model = TfidfVectorizer().fit(X)
        return self

    def transform(self, X):
        return self.tfidf_and_w2v(X, self.tfidf_model)

    def texts_encoder(self, texts):
        with tf.Graph().as_default():
            embed = hub.Module("https://tfhub.dev/google/nnlm-ja-dim128/1")
            embeddings = embed(texts)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                sess.run(tf.tables_initializer())
                result = sess.run(embeddings)
        return result

    def tfidf_and_w2v(self, X, tfidf_model):
        tmp = tfidf_model.transform(X)
        a = tmp.toarray()
        if isinstance(X, pd.core.series.Series):
            X = X.tolist()
        b = self.texts_encoder(X)
        return np.hstack((a, b))

pipe = Pipeline([('vector',TfIdfWord2VecVectorizer()), ("clf", LogisticRegression())])
np.mean(cross_val_score(pipe, X, y, cv=3))

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e68

0.3971239121898487

In [152]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [153]:
pipe.fit(X_train, y_train)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


Pipeline(memory=None,
     steps=[('vector', <__main__.TfIdfWord2VecVectorizer object at 0x7f04835605c0>), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [154]:
y_pred = pipe.predict(X_test)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


In [155]:
from sklearn.metrics import classification_report
emotion_names = ["happy", "sad", "disgust", "angry", "fear", "surprise"]
print(classification_report(y_test, y_pred, target_names=emotion_names))

             precision    recall  f1-score   support

      happy       0.49      0.59      0.54       726
        sad       0.34      0.39      0.37       619
    disgust       0.39      0.48      0.43       730
      angry       0.46      0.42      0.44       756
       fear       0.35      0.28      0.31       711
   surprise       0.31      0.22      0.26       718

avg / total       0.39      0.40      0.39      4260



In [56]:
proba_test = pipe.predict_proba(X_test)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/32f2b2259e1cc8ca58c876921748361283e73997/variables/variables' with embeddings


In [141]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

n = 12000 - 3000
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    df[:n], df[:n]["label"], num_epochs=None, shuffle=True)

predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    df[n:], df[n:]["label"], shuffle=False)

embedded_text_feature_column = hub.text_embedding_column(
        key="text", 
        module_spec="https://tfhub.dev/google/nnlm-ja-dim128/1")

estimator = tf.estimator.DNNClassifier(
        hidden_units=[1200, 600, 150],
        feature_columns=[embedded_text_feature_column],
        n_classes=6,
        optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))

estimator.train(input_fn=train_input_fn, steps=1000);
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)
print("Test set accuracy: {accuracy}".format(**test_eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1v6gxtyk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0488066390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Crea

In [120]:
from sklearn.dummy import DummyClassifier

In [142]:
dpipe = Pipeline([('vector',TfIdfWord2VecVectorizer()), ("clf", DummyClassifier())])
np.mean(cross_val_score(dpipe, X, y, cv=3))

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e68

0.1620901309452671

In [156]:
dpipe.fit(X_train, y_train)
d_y_pred = dpipe.predict(X_test)
print(classification_report(y_test, d_y_pred, target_names=emotion_names))

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
             precision    recall  f1-score   support

      happy       0.16      0.15      0.15       726
        sad       0.16      0.19      0.17       619
    disgust       0.17      0.18      0.17       730
      angry       0.18      0.16      0.17       756
       fear       0.17      0.17      0.17       711
   surprise       0.16      0.16      0.16       718

avg / total       0.17      0.17      0.17      4260



In [157]:
[len(d) for d in data]

[2840, 2840, 2840, 2840, 2840, 2840]

In [161]:
examples, pipe.predict_proba(examples), pipe.decision_function(examples)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


(['やったー うれしい ！ \n', '最悪 ... 死ね \n', 'この 野郎 、 ぶっ た おす \n'],
 array([[0.25946001, 0.13732661, 0.32607777, 0.07504031, 0.04833197,
         0.15376333],
        [0.00487579, 0.11895783, 0.22373823, 0.23420281, 0.37829694,
         0.0399284 ],
        [0.05746742, 0.04949165, 0.22222228, 0.48779324, 0.10251502,
         0.0805104 ]]),
 array([[-1.13207872, -1.90959561, -0.81715024, -2.57896222, -3.04553395,
         -1.77865301],
        [-5.26139413, -1.93749432, -1.17008739, -1.10975529, -0.40359856,
         -3.12057717],
        [-2.74302125, -2.90133036, -1.18653385,  0.05353867, -2.11244961,
         -2.37970112]]))